In [43]:
import pandas as pd
import json
import os

file_root = "data_scraped/"
files = [os.path.join(file_root, json_file) for  json_file in os.listdir(file_root)]

future_df = []

for json_file in files:
  with open(json_file, 'r') as file:
    json_file = json.load(file)

  future_df.append(json_file)

In [44]:
import pandas as pd

df = pd.DataFrame.from_dict(future_df)
df

,url,title,breadcrums,intro,date_verif,text,images,links
0,https://www.service-public.fr/particuliers/vos...,Usurpation de plaque d'immatriculation d'un vé...,"[Accueil, Transports - Mobilité, Carte grise (...",\nSi vous êtes victime d'une usurpation de pla...,Vérifié le 28 janvier 2022,#En ligne\n\n##De quoi s'agit-il ?\n\nL'usurpa...,[https://www.service-public.fr/resources/v-6b4...,"[https://lannuaire.service-public.fr/, https:/..."
1,https://www.service-public.fr/particuliers/vos...,L'administration peut-elle modifier le contrat...,"[Accueil, Travail - Formation, Carrière dans l...",None,Vérifié le 08 novembre 2022,"\nOui, mais uniquement si vous avez été embauc...",[https://www.service-public.fr/resources/v-6b4...,"[https://lannuaire.service-public.fr/, https:/..."
2,https://www.service-public.fr/particuliers/vos...,Impôt sur le revenu - Comment corriger votre d...,"[Accueil, Argent - Impôts - Consommation, Impô...",None,Vérifié le 01 août 2023,"\nEn cas d'erreur ou d'oubli, vous pouvez corr...",[https://www.service-public.fr/resources/v-6b4...,"[https://lannuaire.service-public.fr/, https:/..."
3,https://www.service-public.fr/particuliers/vos...,Les allocations familiales sont-elles saisissa...,"[Accueil, Justice, Saisies et recouvrements, L...",None,Vérifié le 14 octobre 2021,"\nNon, en principe, les prestations familiales...",[https://www.service-public.fr/resources/v-6b4...,[https://www.legifrance.gouv.fr/codes/article_...
4,https://www.service-public.fr/particuliers/vos...,A t-on droit au PTZ quand on est déjà propriét...,"[Accueil, Argent - Impôts - Consommation, Créd...",\nTout dépend de la nature du logement. Si vou...,Vérifié le 21 février 2022,#Propriétaire de votre résidence principale\n\...,[https://www.service-public.fr/resources/v-6b4...,"[https://lannuaire.service-public.fr/, https:/..."
...,...,...,...,...,...,...,...,...
3388,https://www.service-public.fr/particuliers/vos...,Accès aux origines personnelles : peut-on rech...,"[Accueil, Famille - Scolarité, Placement d'un ...","\nOui, les enfants nés à la suite d'un accouch...",Vérifié le 17 août 2023,##Quelles sont les conditions à remplir pour a...,[https://www.service-public.fr/resources/v-6b4...,"[https://lannuaire.service-public.fr/, https:/..."
3389,https://www.service-public.fr/particuliers/vos...,En quoi consiste le plafonnement global des ni...,"[Accueil, Argent - Impôts - Consommation, Impô...",None,Vérifié le 17 avril 2023,\nLe plafonnement global des avantages fiscaux...,[https://www.service-public.fr/resources/v-6b4...,"[https://lannuaire.service-public.fr/, https:/..."
3390,https://www.service-public.fr/particuliers/vos...,"Licenciement économique nul, injustifié ou irr...","[Accueil, Travail - Formation, Licenciement éc...",\nLorsqu'un juge remet en cause un licenciemen...,Vérifié le 07 septembre 2023,#Nul\n\n##Qu'est-ce qu'un licenciement économi...,[https://www.service-public.fr/resources/v-6b4...,"[https://lannuaire.service-public.fr/, https:/..."
3391,https://www.service-public.fr/particuliers/vos...,Comité social et économique (CSE),"[Accueil, Travail - Formation, Représentation ...",\nLe comité social et économique (CSE) est l'i...,Vérifié le 29 septembre 2022,#Entreprise de moins de 11 salariés\n\n\n\n## ...,[https://www.service-public.fr/resources/v-6b4...,"[https://lannuaire.service-public.fr/, https:/..."


In [45]:
import re


def split_with_md_titles(text):
  # Define a regular expression pattern to match Markdown titles, regardless of level
  title_pattern = r'^(#+)\s*(.*?)\s*#*$'

  # Split the text into lines
  lines = text.split('\n')

  # Initialize a list to store title matches or lines that don't match
  title_matches = []

  # Iterate through each line in the Markdown content
  for line in lines:
    # Try to find a match for the title pattern
    match = re.match(title_pattern, line)

    # If a match is found, append it to the list; otherwise, append the original line
    if match:
        title_matches.append(match.groups())  # Append the matched title
    else:
        title_matches.append(('', line))  # Append the original line

  parsed_sections = []
  current_titles = []
  current_content = []

  for title_match in title_matches:
    title_level = len(title_match[0])  # Number of '#' symbols indicates the title level
    title_text = title_match[1].strip()  # Extract and remove leading/trailing spaces

    # Determine if it's a new section (indicated by a higher title level)
    if title_level == 0:
        current_content.append(title_text)
    else:
      if "\n".join(current_content).strip():
        parsed_sections.append([current_titles.copy(), "\n".join(current_content)])

      if len(current_titles) > 0:
        if title_level <= current_titles[-1][0]:
            current_titles = list(filter(lambda x: x[0] < title_level, current_titles))
            current_titles = current_titles if current_titles else []

      current_titles.append((title_level, title_text))
      current_content = []

  # Append the last section to the list
  parsed_sections.append([current_titles, "\n".join(current_content)])

  # Print the parsed sections (titles list and content)
  result = [{
    "subtitles": [title[1] for title in titles],
    "content": content.strip()
  } for titles, content in parsed_sections]

  return result

In [77]:
df2 = df.copy()
df2['paragraph'] = df2['text'].apply(lambda x: split_with_md_titles(x))
df2 = df2.explode('paragraph')
df2 = pd.concat([df2.drop(['paragraph'], axis=1), df2['paragraph'].apply(pd.Series)], axis=1)

/tmp/ipykernel_1409/46906663.py:4: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df2 = pd.concat([df2.drop(['paragraph'], axis=1), df2['paragraph'].apply(pd.Series)], axis=1)


In [91]:
def separate_big_text(row):

    intro_len = 0
    if isinstance(row['intro'], str):
        intro_len = len(row['intro'].split())

    if len(row['content'].split()) + intro_len > 512:
        splitted = row['content'].split('\n')
        midpoint = len(splitted)//2
        part_one = splitted[:midpoint]
        part_two = splitted[midpoint:]
        return ['\n'.join(part_one), '\n'.join(part_two)]
    else:
        return row['content']

df2['content'] = df2.apply(separate_big_text, axis=1)
df2 = df2.explode('content')

In [92]:
def combine_content(row):
    breadcrums = ' > '.join(row["breadcrums"])
    intro = row["intro"]

    subtitles = row["subtitles"]
    subtitles_str = ""
    for i, subtitle in enumerate(subtitles, start=1):
      subtitles_str += "#"*i + subtitle + "\n"

    content = row["content"]

    return f"{breadcrums}\n{intro}\n\n{subtitles_str}\n{content}"

df2["key_content"] = df2.apply(combine_content, axis=1)

In [96]:
df2['key_content_len'] = df2['key_content'].apply(lambda x: len(x.split()))
df2 = df2.sort_values(by='key_content_len', ascending=False)
print(df2.iloc[60:70]['key_content_len'])

664     516
664     516
2753    516
556     515
2306    515
480     514
3149    514
236     513
2785    512
2154    511
Name: key_content_len, dtype: int64


In [ ]:
df2.to_csv("service_public_content_wv.csv")